<a href="https://colab.research.google.com/github/HollowDena/mahina/blob/clean/dp22055.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Autors: *Deniss Petrovs, dp22055*
===========================

#1. Moduļu importēšana

In [1]:
# Importē nepieciešamos moduļus
# Papildiniet pēc nepieciešamības
from google.colab import files
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

#2. Failu ielādēšana

In [18]:
# Izsauc failu augšupielādes dialogu un izvēlas failus Iris_train.csv un Iris_validate.csv
uploaded = files.upload()

# Pārbauda, vai faili ir atrodami Google Colab diskā (darbībām ar failiem var izmantot Bash komandas, priekšā liekot '!')
!ls

Iris_train.csv	Iris_validate.csv  sample_data


#3. Datu sagatavošana

In [2]:
# Ielasa datus pandas dataframe objektos
# training_data tiks izmantoti neironu tīkla apmācīšanai
training_data = pd.read_csv("Iris_train.csv")
# validation_data tiks izmantoti neironu tīkla precizitātes pārbaudīšanai
validation_data = pd.read_csv("Iris_validate.csv")
# test_data izmantos pasniedzējs rezultātu pārbaudei, atstāt aizkomentētu
# test_data = pd.read_csv("Iris_test.csv")

# Pārveido datus neironu tīklam piemērotā formā
# * jūsu kods šeit *
#Nomaina sugu uz float datu tipu
training_data['Species'] = training_data['Species'].replace('Iris-setosa', 0.0)
training_data['Species'] = training_data['Species'].replace('Iris-versicolor', 1.0)
training_data['Species'] = training_data['Species'].replace('Iris-virginica', 2.0)
validation_data['Species'] = validation_data['Species'].replace('Iris-setosa', 0.0)
validation_data['Species'] = validation_data['Species'].replace('Iris-versicolor', 1.0)
validation_data['Species'] = validation_data['Species'].replace('Iris-virginica', 2.0)
validation_data

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,4.4,3.2,1.3,0.2,0.0
1,5.0,3.5,1.6,0.6,0.0
2,5.1,3.8,1.9,0.4,0.0
3,4.8,3.0,1.4,0.3,0.0
4,5.8,2.6,4.0,1.2,1.0
5,5.0,2.3,3.3,1.0,1.0
6,5.6,2.7,4.2,1.3,1.0
7,5.7,3.0,4.2,1.2,1.0
8,5.8,2.7,5.1,1.9,2.0
9,6.8,3.2,5.9,2.3,2.0


In [3]:
# X ir parametri, y ir suga
X = training_data.drop('Species', axis=1)
y = training_data['Species']
X_test = validation_data.drop('Species', axis=1)
y_test = validation_data['Species']
# Pārveido par skaitliskiem masīviem
X = X.values
y = y.values
X_test = X_test.values
y_test = y_test.values

#4. Neironu tīkla definēšana

In [4]:
# Definē neironu tīklu

# * jūsu kods šeit *
torch.manual_seed(42)
class Model(nn.Module):

  def __init__(self, lapas=4, neir1=4, neir2=6, izvads=3):
      super().__init__()
      self.lay1 = nn.Linear(lapas, neir1)
      self.lay2 = nn.Linear(neir1, neir2)
      self.izv = nn.Linear(neir2, izvads)
  def forward(self, x):
      x = F.relu(self.lay1(x))
      x = F.relu(self.lay2(x))
      x = self.izv(x)
      return x


In [5]:
model = Model()

#5. Neironu tīkla apmācīšana

In [6]:
# Apmāca neironu tīklu klasificēt īrisus pēc to ziedu raksturlielumiem
# * jūsu kods šeit *
#Uztaisa tenzorus
X_train = torch.FloatTensor(X)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y)
y_test = torch.LongTensor(y_test)

In [7]:
#Izvēlas kļūdas noteikšanas veidu
crit = nn.CrossEntropyLoss()
#izvēlas optimizatoru
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [8]:
#Apmācības cikls
cikli = 100
kluudas= []
for i in range(cikli):
  #Iet cauri cikliem un min
  y_pred = model.forward(X_train)
  #Kļūdas noteikšana
  kluuda = crit(y_pred, y_train)
  #Pievieno kļūdu kļūdu masīvam
  kluudas.append(kluuda.detach().numpy)
  if i%10 ==0:
    print(f'Cikls: {i} un kļūda: {kluuda}')
  #Nobalansē "svarus"
  optimizer.zero_grad()
  kluuda.backward()
  optimizer.step()

Cikls: 0 un kļūda: 1.5109881162643433
Cikls: 10 un kļūda: 1.0473606586456299
Cikls: 20 un kļūda: 0.9031420350074768
Cikls: 30 un kļūda: 0.7526842951774597
Cikls: 40 un kļūda: 0.5962424874305725
Cikls: 50 un kļūda: 0.4871203303337097
Cikls: 60 un kļūda: 0.4132227301597595
Cikls: 70 un kļūda: 0.3504502773284912
Cikls: 80 un kļūda: 0.2879679203033447
Cikls: 90 un kļūda: 0.2286035716533661


# 6. Neironu tīkla pārbaudīšana.
Jāizdrukā visu pārbaužu rezultāti viegli saprotamā formā (12 rindiņas, pa vienai katram validācijas kopas piemēram), kā arī kopējais precizitātes novērtējums.  
 Noformējuma piemērs:  
 1. rindiņa: klasifikatora rezultāts: Iris-setosa; reālā suga: Iris-setosa, pareizi  
 ...
 12. rindiņa: klasifikatora rezultāts: Iris-versicolor; reālā suga: Iris-virginica, nepareizi  
 Precizitāte: 58% (7/12)

In [11]:
# Pārbauda neirona tīkla precizītāti ar validācijas datu kopu

# * jūsu kods šeit *
#Izslēdz svaru balansēšanu
with torch.no_grad():
  y_eval = model.forward(X_test)
  kluuda = crit(y_eval, y_test)

In [10]:
pareizi = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)
#Pārveidojam sugas cilvēkam saprotamā valodā
    if y_test[i] == 0:
      x = "Iris-setosa"
    elif y_test[i] == 1:
      x = "Iris-versicolor"
    elif y_test[i] == 2:
      x = "Iris-virginica"
    if y_val.argmax() == 0:
      z = "Iris-setosa"
    if y_val.argmax() == 1:
      z = "Iris-versicolor"
    if y_val.argmax() == 2:
      z = "Iris-virginica"


#Izdrukājam visus pareizos
    print(f'{i+1}.) {str(y_val)} Modelis atpazina {x}, pareizā atbilde {z} ')

    if y_val.argmax().item() == y_test[i]:
      pareizi+=1
print(f'Pareizi tika atpazīti {pareizi} no 12 īrisiem')

1.) tensor([ 4.2416,  0.6053, -7.6394]) Modelis atpazina Iris-setosa, pareizā atbilde Iris-setosa 
2.) tensor([ 4.0963,  0.7313, -7.6125]) Modelis atpazina Iris-setosa, pareizā atbilde Iris-setosa 
3.) tensor([ 4.3739,  0.7153, -8.0471]) Modelis atpazina Iris-setosa, pareizā atbilde Iris-setosa 
4.) tensor([ 4.0191,  0.6417, -7.3805]) Modelis atpazina Iris-setosa, pareizā atbilde Iris-setosa 
5.) tensor([-1.5273,  1.9862,  0.0423]) Modelis atpazina Iris-versicolor, pareizā atbilde Iris-versicolor 
6.) tensor([-0.6851,  1.6451, -0.9727]) Modelis atpazina Iris-versicolor, pareizā atbilde Iris-versicolor 
7.) tensor([-1.9073,  2.0403,  0.6340]) Modelis atpazina Iris-versicolor, pareizā atbilde Iris-versicolor 
8.) tensor([-1.2056,  1.9555, -0.4681]) Modelis atpazina Iris-versicolor, pareizā atbilde Iris-versicolor 
9.) tensor([-4.3972,  2.5715,  4.2648]) Modelis atpazina Iris-virginica, pareizā atbilde Iris-virginica 
10.) tensor([-5.3530,  2.9984,  5.3776]) Modelis atpazina Iris-virginic